## 1. color mapping
#### context engineering 전, dummy를 가지고 색 선정 함수를 정의하고자함.

In [59]:
import pandas as pd

- KOTE 44가지 감정의 긍/부정/중립 기준 적용

In [55]:
positive_emotions = ['감동/감탄', '고마움', '기대감', '기쁨', '뿌듯함',
'신기함/관심', '아껴주는', '안심/신뢰', '존경', '즐거움/신남', '편안/쾌적', '행복', '환영/호의',
'흐뭇함(귀여움/예쁨)',
]
negative_emotions = ['경악', '공포/무서움', '귀찮음', '당황/난처', '부끄러움', '부담/안_내킴', '불쌍함/연민', '불안/걱정', 
                     '불평/불만', '슬픔', '서러움', '안타까움/실망', '어이없음', '역겨움/징그러움', '의심/불신', '짜증', 
                     '재미없음', '절망', '죄책감', '증오/혐오', '지긋지긋', '패배/자기혐오', '한심함', '화남/분노', '힘듦/지침']
neutral_emotions = ['깨달음', '놀람', '비장함', '우쭐댐/무시함']

no_emotion = ['없음']


In [89]:
result_emotions = {'불쌍함/연민': 0.44,
'불안/걱정': 0.56,
'서러움': 0.51,
'슬픔': 0.7,
'아껴주는' : 0.47,
'안타까움/실망': 0.31,
'없음': 0.8,
'기쁨':0.6
}

result_IRI = '맑은'

In [90]:
# 감정 출력 값 전처리 : 없음을 제거, 감정 범주화에 해당하지 않는 감정 제거 ex) 한심함
def filter_emotions(result_emotions, result_IRI):
    # Filter out the '없음' key
    filtered_emotions = {key: value for key, value in result_emotions.items() if key != '없음'}
    
    # Sort emotions by their values in descending order
    sorted_emotions = sorted(filtered_emotions.items(), key=lambda item: item[1], reverse=True)
    
    # Extract the keys of the top two emotions
    top_two_emotions = [sorted_emotions[0][0], sorted_emotions[1][0], result_IRI] if len(sorted_emotions) >= 2 else []
    
    return top_two_emotions

In [91]:
# Example usage
## 주-보조 감정은 감정 분류 모델(KPoEM)에서, 형용사는 LLM모델이 반환.
top_emotions = filter_emotions(result_emotions, result_IRI)
print(top_emotions)  # Output: ['슬픔', '기쁨']

['슬픔', '기쁨', '맑은']


In [123]:
# CSV 파일 경로
file_path = "korean emtion-color dataset 2.0 - SUM.csv"
# 파일 읽기 (처음엔 컬럼명을 무시하고 불러오기)
df_raw = pd.read_csv(file_path, header=None)

# 첫 번째 행을 컬럼명으로 지정
df_raw.columns = df_raw.iloc[1]
df = df_raw[2:]  # 첫 행은 이제 컬럼으로 사용했으니 제외

In [96]:
# Function to categorize the emotion
def categorize_emotion(primary_emotion, secondary_emotion):
    # 감정 사전 정의 (예시, 필요에 따라 확장 가능)

    # 내부 분류 함수
    def classify(emotion):
        if emotion in positive_emotions:
            return "positive"
        elif emotion in negative_emotions:
            return "negative"
        elif emotion in neutral_emotions:
            return "neutral" # 아직 처리 안함.
        else:
            return "no_emotion" #아직 처리 안함.
    
    def handle_neutral_case(primary_emotion, secondary_emotion):
        # 감정이 'neutral'인 경우는 별도의 로직으로 처리
        if primary_emotion == "neutral" or secondary_emotion == "neutral":
            return "배색3"

    # 각각 분류
    primary_category = classify(primary_emotion)
    secondary_category = classify(secondary_emotion)
    
    # 'neutral'이 포함된 경우는 별도 함수로 처리
    if "neutral" in [primary_category, secondary_category] :
        print(f"Primary: {primary_category}, Secondary: {secondary_category}")
        return handle_neutral_case(primary_category, secondary_category)
    print(f"Primary: {primary_category}, Secondary: {secondary_category}")

    # 조건 비교 후 mode 반환
    if primary_category == secondary_category in ["positive", "negative"]:
        return "유사색상"
    else:
        return "대비색상"



In [97]:
# Example usage
print(categorize_emotion(top_emotions[0],top_emotions[1]))

Primary: negative, Secondary: positive
대비색상


In [171]:
def categorize_colors(color):
    munsell_colors = ['R', 'YR', 'Y', 'GY', 'G', 'BG', 'B', 'PB', 'P', 'RP']
    total = len(munsell_colors)

    if color not in munsell_colors:
        return "unknown_color"

    idx = munsell_colors.index(color)

    # 유사색상군
    similar_1_idx = [(idx - 1) % total, (idx + 1) % total]
    similar_2_idx = [(idx - 2) % total, (idx + 2) % total]

    # 대비색상군
    contrast_2_idx = [(idx - 3) % total, (idx + 3) % total]
    contrast_1_idx = [(idx - 4) % total, (idx + 4) % total]
    contrast_idx = [(idx - 5) % total]  # 보색 (정반대 5칸 거리)

    # 변환
    similar_colors_1 = [munsell_colors[i] for i in similar_1_idx]
    similar_colors_2 = [munsell_colors[i] for i in similar_2_idx]
    contrast_colors_2 = [munsell_colors[i] for i in contrast_2_idx]
    contrast_colors_1 = [munsell_colors[i] for i in contrast_1_idx]
    contrast_color = [munsell_colors[i] for i in contrast_idx]

    return {
        "center_color": color,
        "similar_colors_1": similar_colors_1,
        "similar_colors_2": similar_colors_2,
        "contrast_colors_1": contrast_colors_1,
        "contrast_colors_2": contrast_colors_2,
        "contrast_color": contrast_color  # 보색
    }
        

In [236]:
def define_colors(primary_emotion, secondary_emotion, df):
    mode_colors = categorize_emotion(primary_emotion, secondary_emotion)
    top_ranked_value = df[df['감정 어휘'] == primary_emotion].sort_values(by='평가 순위').iloc[0].to_dict()
    primary_color = categorize_colors(top_ranked_value['색상약호'])
    
    # 유사색상 모드일 때
    if mode_colors == "유사색상":
        secondary_ranked_value = df[df['감정 어휘'] == secondary_emotion].sort_values(by='평가 순위').iloc[0].to_dict()
        print(secondary_ranked_value)
        
    # 대비색상 모드일 때
    elif mode_colors == "대비색상": 
        secondary_ranked_df = df[df['감정 어휘'] == secondary_emotion]
        
        contrast_matches_1 = []
        contrast_matches_2 = []
        contrast_match_exact = []

                
        # contrast_colors_1 체크
        for contrast_color in primary_color['contrast_colors_1']:
            matched = secondary_ranked_df[secondary_ranked_df['색상약호'] == contrast_color]
            if not matched.empty:
                contrast_matches_1.append(matched)

        # contrast_colors_2 체크
        for contrast_color in primary_color['contrast_colors_2']:
            matched = secondary_ranked_df[secondary_ranked_df['색상약호'] == contrast_color]
            if not matched.empty:
                contrast_matches_2.append(matched)
                
        # contrast_color (보색) 체크
        contrast_color = primary_color['contrast_color'][0]  # 단일 값 리스트
        matched_exact = secondary_ranked_df[secondary_ranked_df['색상약호'] == contrast_color]
        if not matched_exact.empty:
            contrast_match_exact.append(matched_exact)

        #출력
        print("대비색상군1과 일치하는 색상 ↓")
        print(pd.concat(contrast_matches_1) if contrast_matches_1 else "대비색상군1 없음")

        print("\n대비색상군2와 일치하는 색상 ↓")
        print(pd.concat(contrast_matches_2) if contrast_matches_2 else "대비색상군2 없음")

        print("\n보색과 정확히 일치하는 색상 ↓")
        print(pd.concat(contrast_match_exact) if contrast_match_exact else "보색 일치 항목 없음")
        
    #보색이 정확히 일치하는 경우
    if contrast_match_exact:
        contrast_dicts = contrast_match_exact[0].to_dict(orient='records')
        return min(contrast_dicts, key=lambda x: x['평가 순위']) # '평가 순위'가 가장 높은 값 (숫자가 클수록 높은 순위라면 ↓)
    
    # 대비색상군1과 대비색상군2 중 하나라도 일치하는 경우.
    if contrast_matches_1 or contrast_matches_2: 
        if contrast_matches_1: # 대비색상군1을 우선 선정.
            contrast_matches_1_df = pd.concat(contrast_matches_1).to_dict(orient='records')
            print(contrast_matches_1_df)
            return min(contrast_matches_1_df, key=lambda x: x['평가 순위'])
        else: #대비색상군1이 없고, 대비색상군2가 있는 경우
            contrast_matches_2_df = pd.concat(contrast_matches_2).to_dict(orient='records')
            print(contrast_matches_2_df)
            return min(contrast_matches_2_df, key=lambda x: x['평가 순위'])
        
    # 예외처리 : 대비색상, 대비색상군1, 대비색상군2가 전부 없는 경우
    

    

In [237]:
color = define_colors(top_emotions[0],top_emotions[1], df) # 주감정과 보조감정, KCoED 데이터셋
color


Primary: negative, Secondary: positive
대비색상군1과 일치하는 색상 ↓
대비색상군1 없음

대비색상군2와 일치하는 색상 ↓
1  감정id 감정 어휘 평가 순위    R    G    B   샘플  \
95   17    기쁨     4  208   47   72  NaN   
96   17    기쁨     5  239  108  112  NaN   
97   17    기쁨     6  231  213  212  NaN   

1                                             출처/참고문헌  \
95  김애경,and 오윤경. "긍정 및 부정 정서어휘에 대한 색 반응 연구." 조형미디어...   
96  김애경,and 오윤경. "긍정 및 부정 정서어휘에 대한 색 반응 연구." 조형미디어...   
97  김애경,and 오윤경. "긍정 및 부정 정서어휘에 대한 색 반응 연구." 조형미디어...   

1                                    출처/참고문헌2  색상 색상약호 색상 온도  NaN  
95  https://gist.github.com/hidex7777/5268703  빨강    R  Warm  NaN  
96  https://gist.github.com/hidex7777/5268704  빨강    R  Warm  NaN  
97  https://gist.github.com/hidex7777/5268705  빨강    R  Warm  NaN  

보색과 정확히 일치하는 색상 ↓
1  감정id 감정 어휘 평가 순위    R    G    B   샘플  \
92   17    기쁨     1  238  201    0  NaN   
93   17    기쁨     2  250  216   49  NaN   
94   17    기쁨     3  239  230  198  NaN   

1                                             출처

{'감정id': '17',
 '감정 어휘': '기쁨',
 '평가 순위': '1',
 'R': '238',
 'G': '201',
 'B': '0',
 '샘플': nan,
 '출처/참고문헌': '김애경,and 오윤경. "긍정 및 부정 정서어휘에 대한 색 반응 연구." 조형미디어학 19.1 (2016): 59-66.',
 '출처/참고문헌2': 'https://gist.github.com/hidex7777/5268700',
 '색상': '노랑',
 '색상약호': 'Y',
 '색상 온도': 'Warm',
 nan: nan}